In [26]:
import pandas as pd
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
# spark_version = 'spark-3.<enter version>'
# os.environ['SPARK_VERSION']=spark_version
# # Install Spark and Java
# !apt-get update
# !apt-get install openjdk-11-jdk-headless -qq > /dev/null
# !wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
# !tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
# # Set Environment Variables
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
# # Start a SparkSession
# import findspark
# findspark.init()
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("Q1").getOrCreate()

In [27]:
# Mapping lets you format an entire DataFrame
products = "Data/products.csv"
products_df = pd.read_csv(products)
products_df.head()

,brand,key,name,subhead,description,rating,rating_count,ingredients
0,bj,0_bj,Salted Caramel Core,Sweet Cream Ice Cream with Blonde Brownies & a...,Find your way to the ultimate ice cream experi...,3.7,208,"CREAM, SKIM MILK, LIQUID SUGAR, WATER, BROWN S..."
1,bj,1_bj,Netflix & Chilll'd™,Peanut Butter Ice Cream with Sweet & Salty Pre...,There’s something for everyone to watch on Net...,4.0,127,"CREAM, SKIM MILK, LIQUID SUGAR, WATER, SUGAR, ..."
2,bj,2_bj,Chip Happens,A Cold Mess of Chocolate Ice Cream with Fudge ...,Sometimes “chip” happens and everything’s a me...,4.7,130,"CREAM, LIQUID SUGAR, SKIM MILK, WATER, SUGAR, ..."
3,bj,3_bj,Cannoli,Mascarpone Ice Cream with Fudge-Covered Pastry...,As a Limited Batch that captured the rapture o...,3.6,70,"CREAM, SKIM MILK, LIQUID SUGAR, WATER, CORN S..."
4,bj,4_bj,Gimme S’more!™,Toasted Marshmallow Ice Cream with Chocolate C...,It’s a gimme: there’s always room for s’more. ...,4.5,281,"CREAM, SKIM MILK, WATER, LIQUID SUGAR, SUGAR, ..."


In [31]:
products_trim_df = products_df.drop(columns=['description'])
# products_trim_df=products_trim_df.reset_index(drop=True, inplace=True)
products_trim_df.to_csv("Data/products_trim.csv", index=False)

In [37]:
ingredient_list = products_trim_df.ingredients.unique()
unique_ingredient_dict = {}
    # Iterate through the outer list
for ingredient in ingredient_list:
    i = ingredient.split(', ')
    for i_list in i:
        if i_list != "":
            if i_list not in unique_ingredient_dict:
                    unique_ingredient_dict[i_list]=1
            else:
                    unique_ingredient_dict[i_list]=unique_ingredient_dict[i_list]+1
print(unique_ingredient_dict)
    

{'CREAM': 250, 'SKIM MILK': 162, 'LIQUID SUGAR': 60, 'WATER': 132, 'BROWN SUGAR': 28, 'SUGAR': 309, 'MILK': 115, 'WHEAT FLOUR': 75, 'EGG YOLKS': 126, 'CORN SYRUP': 140, 'EGGS': 27, 'BUTTER': 71, 'BUTTEROIL': 12, 'PECTIN': 62, 'SEA SALT': 16, 'SOYBEAN OIL': 58, 'VANILLA EXTRACT': 118, 'GUAR GUM': 82, 'SOY LECITHIN': 158, 'BAKING POWDER': 2, 'BAKING SODA': 72, 'SALT': 215, 'CARRAGEENAN': 63, 'LACTASE': 4, 'PEANUTS': 31, 'CANOLA OIL': 29, 'CORN STARCH': 28, 'PEANUT OIL': 27, 'COCOA POWDER': 24, 'INVERT CANE SUGAR': 4, 'MILK FAT': 26, 'EGG WHITES': 18, 'TAPIOCA STARCH': 16, 'BARLEY MALT': 5, 'MALTED BARLEY FLOUR': 6, 'COCOA': 99, 'POTATO': 1, 'COCONUT OIL': 145, 'CORN SYRUP SOLIDS': 3, 'RICE STARCH': 1, 'SUNFLOWER OIL': 12, 'YEAST EXTRACT': 1, 'NATURAL FLAVOR': 165, 'MILKFAT': 16, 'ENZYMES': 3, 'BAKING SODA. CONTAINS MILK': 1, 'WHEAT AND SOY': 1, ' WATER': 1, 'DRIED CANE SYRUP': 21, 'BUTTER OIL': 9, 'NATURAL FLAVORS': 28, 'LOCUST BEAN GUM': 1, 'CITRIC ACID': 14, 'VANILLA BEAN SEEDS': 1, 'L

In [42]:
sorted(unique_ingredient_dict.items(),key=lambda x:x[1],reverse=True)

[('SUGAR', 309),
 ('CREAM', 250),
 ('SALT', 215),
 ('NATURAL FLAVOR', 165),
 ('SKIM MILK', 162),
 ('SOY LECITHIN', 158),
 ('COCONUT OIL', 145),
 ('CORN SYRUP', 140),
 ('WATER', 132),
 ('EGG YOLKS', 126),
 ('VANILLA EXTRACT', 118),
 ('MILK', 115),
 ('COCOA', 99),
 ('GUAR GUM', 82),
 ('WHEAT FLOUR', 75),
 ('BAKING SODA', 72),
 ('BUTTER', 71),
 ('CARRAGEENAN', 63),
 ('PECTIN', 62),
 ('CHOCOLATE', 62),
 ('WHEY', 62),
 ('VEGETABLE GUMS', 62),
 ('LIQUID SUGAR', 60),
 ('SOYBEAN OIL', 58),
 ('COCOA BUTTER', 57),
 ('MONO AND DIGLYCERIDES', 56),
 ('CAROB BEAN', 50),
 ('GUAR', 49),
 ('COCOA PROCESSED WITH ALKALI', 45),
 ('DEXTROSE', 45),
 ('CAROB BEAN GUM', 44),
 ('MOLASSES', 35),
 ('PEANUTS', 31),
 ('VITAMIN A PALMITATE', 30),
 ('DUTCHED COCOA', 30),
 ('CANOLA OIL', 29),
 ('CANE SUGAR', 29),
 ('BROWN SUGAR', 28),
 ('CORN STARCH', 28),
 ('NATURAL FLAVORS', 28),
 ('EGGS', 27),
 ('PEANUT OIL', 27),
 ('MILK FAT', 26),
 ('PALM OIL', 26),
 ('COCOA POWDER', 24),
 ('ANNATTO', 24),
 ('TARA', 24),
 ('LEMO

In [14]:
len(final_ingredient_list)

414